In [2]:
import pandas as pd
import pandas_gbq as pgbq
from scipy import stats

google_project_id = 'solid-ridge-104914'

In [8]:
# Calculate network properties 

with open('../sql/cumulative_networks/praise_net.txt') as query_file:
    praise_query = query_file.read()

praise_net = pgbq.read_gbq(praise_query, google_project_id, dialect='standard')
pgbq.to_gbq(praise_net, 'community_networks.praise_net', google_project_id, if_exists='replace')

Requesting query... ok.
Query running...
  Elapsed 11.88 s. Waiting...
  Elapsed 22.43 s. Waiting...
  Elapsed 32.77 s. Waiting...
Query done.
Processed: 19.5 GB

Retrieving results...
  Got page: 1; 100% done. Elapsed 41.6 s.
Got 2023 rows.

Total time taken 41.69 s.
Finished at 2017-06-24 23:11:21.


NameError: name 'transaction_net' is not defined

In [10]:
pgbq.to_gbq(praise_net, 'community_networks.praise_net', google_project_id, if_exists='replace')




Streaming Insert is 100.0% Complete




In [9]:
# Cross correlate community facts and output key correlations

variables = list(praise_net)[3:len(praise_net)] #this is not a good way to create the list of variables

praise_corr_matrix = pd.DataFrame(index=variables, columns=variables)
praise_key_correlations = pd.DataFrame(columns=['pair','coefficient'])
corr_checked = list()

i = 0
j = 0

for i in range(len(variables)):
    for j in range(len(variables)):
        pair = variables[i]+','+variables[j]
        rho, pval = stats.spearmanr(praise_net[[variables[i],variables[j]]])
        praise_corr_matrix.set_value(variables[i], variables[j], rho)
        if pair not in corr_checked and rho < 0.99 and (rho > 0.5 or rho < -0.5):
            praise_key_correlations.loc[len(praise_key_correlations)] = [pair, rho]
            corr_checked.append(variables[j]+','+variables[i])

praise_key_correlations.sort_values(by='coefficient', ascending=False, inplace=True)
praise_key_correlations.reset_index(drop=True, inplace=True)

#Save to BigQuery
pgbq.to_gbq(praise_corr_matrix, 'community_networks.praise_corr_matrix', google_project_id, if_exists='replace')
pgbq.to_gbq(praise_key_correlations, 'community_networks.praise_key_correlations', google_project_id, if_exists='replace')
praise_key_correlations




Streaming Insert is 100.0% Complete





Streaming Insert is 100.0% Complete




,pair,coefficient
0,"nodes,mau_may_2017",0.893027
1,"nodes,inventory",0.867513
2,"items_sold_may_2017,mau_may_2017",0.837955
3,"mau_may_2017,avg_indegree",0.821748
4,"nodes,items_sold_may_2017",0.807662
5,"items_sold_may_2017,inventory",0.770706
6,"inventory,mau_may_2017",0.748851
7,"avg_degree,avg_indegree",0.735901
8,"nodes,avg_indegree",0.720257
9,"inventory,avg_indegree",0.694109


In [11]:
praise_corr_matrix

,nodes,items_sold_may_2017,items_sold_change_2017,inventory,mau_may_2017,mau_change_2017,avg_weight,network_density_X_100,avg_degree,avg_indegree,avg_outdegree,indegree_skew
nodes,1,0.807662,0.151469,0.867513,0.893027,0.214864,0.586572,-0.754622,0.300128,0.720257,0.129705,0.381074
items_sold_may_2017,0.807662,1,0.366256,0.770706,0.837955,0.327119,0.458644,-0.549633,0.344998,0.673802,0.051633,0.447587
items_sold_change_2017,0.151469,0.366256,1,0.0983015,0.224724,0.250215,0.108787,-0.0486455,0.149486,0.160353,0.0410033,0.0512988
inventory,0.867513,0.770706,0.0983015,1,0.748851,0.0465965,0.518662,-0.656455,0.266542,0.694109,0.0523586,0.463889
mau_may_2017,0.893027,0.837955,0.224724,0.748851,1,0.491298,0.562033,-0.488338,0.554223,0.821748,0.182491,0.394245
mau_change_2017,0.214864,0.327119,0.250215,0.0465965,0.491298,1,0.161439,0.067377,0.410765,0.294453,0.199944,0.013927
avg_weight,0.586572,0.458644,0.108787,0.518662,0.562033,0.161439,1,-0.282295,0.45657,0.63093,0.327586,0.104028
network_density_X_100,-0.754622,-0.549633,-0.0486455,-0.656455,-0.488338,0.067377,-0.282295,1,0.351872,-0.21983,0.29456,-0.445968
avg_degree,0.300128,0.344998,0.149486,0.266542,0.554223,0.410765,0.45657,0.351872,1,0.735901,0.612612,-0.0215053
avg_indegree,0.720257,0.673802,0.160353,0.694109,0.821748,0.294453,0.63093,-0.21983,0.735901,1,0.287032,0.383972


### Notes on Correlations among transaction network properties
 
OBSERVATIONS
 
 Inventory correlations:
 
 - Communities where the average seller has a high number of buyers are also likely more have more inventory (0.658)
 - density is inversly correlated with inventory (row 17, -0.596)